<a href="https://colab.research.google.com/github/HandyGunawan17/Portfolio_Management/blob/main/Paper_Replication/MSAR_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The first code of the series of Markov Switching autoregressive model.

This code studies and replicate statsmodel Markov Switching (MS) Autoregressive (AR) model.

It Consists of Markov Switching Model, Markov Regression and finally Markov_auto Regression

 # First, we do the MArkov Switching.

In [1]:
import warnings

import numpy as np
import pandas as pd
from scipy.special import logsumexp

# ! pip install --upgrade Cython
# ! pip install --upgrade git+https://github.com/statsmodels/statsmodels

from statsmodels.tools.tools import Bunch
from statsmodels.tools.numdiff import approx_fprime_cs, approx_hess_cs
from statsmodels.tools.decorators import cache_readonly
from statsmodels.tools.eval_measures import aic, bic, hqic
from statsmodels.tools.tools import pinv_extended
from statsmodels.tools.sm_exceptions import EstimationWarning

import statsmodels.base.wrapper as wrap
from statsmodels.base.data import PandasData

import statsmodels.tsa.base.tsa_model as tsbase
from statsmodels.tsa.statespace.tools import find_best_blas_type, prepare_exog

from statsmodels.tsa.regime_switching._hamilton_filter import (
    shamilton_filter_log, dhamilton_filter_log, chamilton_filter_log,
    zhamilton_filter_log)
from statsmodels.tsa.regime_switching._kim_smoother import (
    skim_smoother_log, dkim_smoother_log, ckim_smoother_log, zkim_smoother_log)

prefix_hamilton_filter_log_map = {
    's': shamilton_filter_log, 'd': dhamilton_filter_log,
    'c': chamilton_filter_log, 'z': zhamilton_filter_log
}

prefix_kim_smoother_log_map = {
    's': skim_smoother_log, 'd': dkim_smoother_log,
    'c': ckim_smoother_log, 'z': zkim_smoother_log
}


In [28]:
def _logistic(x):
    """
    Note that this is not a vectorized function
    """
    x = np.array(x)
    # np.exp(x) / (1 + np.exp(x))
    if x.ndim == 0:
        y = np.reshape(x, (1, 1, 1))
    # np.exp(x[i]) / (1 + np.sum(np.exp(x[:])))
    elif x.ndim == 1:
        y = np.reshape(x, (len(x), 1, 1))
    # np.exp(x[i,t]) / (1 + np.sum(np.exp(x[:,t])))
    elif x.ndim == 2:
        y = np.reshape(x, (x.shape[0], 1, x.shape[1]))
    # np.exp(x[i,j,t]) / (1 + np.sum(np.exp(x[:,j,t])))
    elif x.ndim == 3:
        y = x
    else:
        raise NotImplementedError

    tmp = np.c_[np.zeros((y.shape[-1], y.shape[1], 1)), y.T].T
    print(tmp)
    print(y)
    evaluated = np.reshape(np.exp(y - logsumexp(tmp, axis=0)), x.shape)

    return evaluated

(1.6094379124341005, 1.6094379124341003)